# DICOM to BIDS converter

Welcome to this DICOM to BIDS converter!

This converter only handles the following MRI sequences:
    - MPRAGE
    - FLAIR
    - EPI
    - Phase
    - Diffusion weighted imaging

The converter should be used on dicom files coming directly out of the MR machine.

#### /!\ Important /!\ : Run steps 1 and 2 before running anything else

Author: Maxence Wynen -- maxencewynen@gmail.com
        Colin Vanden Bulcke -- colin.vandenbulcke@uclouvain.be

## 1. Import dependencies

In [1]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 
from dicom2bids import *
from change_orientation import *
from reorient_phase_unwrapped import *
from corr_header_for_registration import *
print("[INFO] Imports done")
#"C:/Users/maxen/OneDrive/Bureau/UCLouvain/Q14/WSBIM2243/project/WSBIM2243/database"
# Path to dcm2niix.exe converter -- Only for windows(Comment for unix/macOS users) 
dicom2niix_path = "dcm2niix" 
# dicom2niix_path = "C:/Users/maxen/OneDrive/Bureau/UCLouvain/Q14/WSBIM2243/project/dcm2niix.exe" # Uncomment for Windows

[INFO] Imports done


## 2. Set paths

In [2]:
# Output BIDS directory. Should previously be created.
# Root BIDS directory where subject folders and derivative folder are located
bids_dir = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels"


## 3. Automatically import a DICOM directory into a BIDS format

In [14]:
# Path to dicom directory
directory = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/DICOM/sub-059/sub-059_ses-01/GEVENOIS_3992098701"

# Convert all DICOMs
dicom_series = convert_all_dicoms(directory, dicom2niix_path)

# Create directories in the BIDS file structure by giving an incremental id
# pat_id, session = make_directories(bids_dir,pat_id=None,session=None)
# To specify the patient id:
pat_id, session = make_directories(bids_dir,pat_id='059',session='01')
# To specify the patient id and session:
# pat_id, session = make_directories(bids_dir,pat_id='ID_TO_SPECIFY',session='SESSION_TO_SPECIFY')

# Rename and move all (interesting) converted files into the bids directory
rename_and_move_nifti(dicom_series, bids_dir, pat_id, session)

print("[INFO] Done")

[INFO] Starting to convert ...
SUBDIR: /media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/DICOM/sub-059/sub-059_ses-01/GEVENOIS_3992098701	DIRS: []


AttributeError: 'DicomDir' object has no attribute 'SeriesDescription'

In [13]:
delete_nii_json_in_dicomdir(dicom_series)

## 4.Optional functionalities

### 4.1. Delete subject

In [11]:
# Remove a patient from the database
delete_subject(bids_dir, "2097")

### 4.2. Delete session

In [5]:
# Remove a particular session from a particular patient
delete_session(bids_dir, "026", "03")
print(bids_dir)

/media/maggi/Elements/BACKUP/DISSECT_MS_DATABASE


### 4.3 Rename subject

In [10]:
# Rename a subject in the database
rename_subject(bids_dir, old_id='1000',new_id='2000')

FileNotFoundError: Subject 1000 is not in the database.

### 4.4. Reorient an image

In [24]:
EPI_magnitude = 'acq-mag_T2star'
EPI_phase = 'acq-phase_T2star'
phase_wrapped = 'phase_WRAPPED'
T2 = 'T2'
T1 = 'T1'
T1_Gd = 'T1w_Gd'
FLAIRstar = 'acq-star_FLAIR'
QSM = 'QSM'

reorient(subject = '059', session = '02', 
         sequence_name = EPI_magnitude + '_matlab', 
         axis = 'z',
         DIR=bids_dir)

In [17]:
##### CHANGE AFFINE ######
replace_affine_or_header(path_to_img = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/sub-059/ses-01/anat/sub-059_ses-01_acq-phase_T2star_UNWRAPPED.nii.gz", 
                         path_to_target = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/sub-137/ses-01/anat/sub-137_ses-01_acq-mag_T2star.nii.gz", affine=True, header=False)


### 4.5. Reorient phase image after docker unwrapping

In [15]:
### REORIENT THE UNWRAPPED PHASE IMAGE AFTER DOCKER UNWRAPPING ###
phase_filename = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/sub-059/ses-01/anat/sub-059_ses-01_acq-phase_T2star_UNWRAPPED.nii.gz"
t2star_filename = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/sub-059/ses-01/anat/sub-059_ses-01_acq-mag_T2star.nii.gz"

reorient_phase_unwrapped(phase_filename, t2star_filename)

[INFO] Phase Reoriented !


In [20]:
### REORIENT THE UNWRAPPED PHASE IMAGE AFTER DOCKER UNWRAPPING (second possibility) ###
phase_filename = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/sub-026/ses-01/anat/sub-026_ses-01_phase.nii.gz"

reorient_phase_unwrapped_2(phase_filename)

TypeError: can only concatenate list (not "str") to list

### 4.6. Correct header for registration

In [76]:
### 4.6. Correct header for registration ###
filename = "/media/maggi/MS-PRL/MS-PRL/MS-PRL_Brussels/sub-058/ses-02/anat/sub-058_ses-02_acq-phase_T2star.nii.gz"
corr_header_for_registration(filename)

### 4.7. Separate 4D image EPI

In [24]:
separate_epi(subject='126', session='01', 
             DIR=bids_dir)